In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from skimage import io
from skimage.color import rgb2gray
import matplotlib

In [ ]:
df = pd.read_csv("HistoQuant/muscle_atlas_2_7_filt_triple.csv")
df_he = df[df["Staining method"] == "HE"]
df_he.head(1)

In [ ]:
file = df_he.iloc[138,1]
calibration = df_he.iloc[138,11]
THRESHOLD = 0.25

In [ ]:
filename = os.path.join("HistoQuant/raw", file)

img = io.imread(filename)
grayscale = rgb2gray(img)
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
ax = axes.ravel()

ax[0].imshow(img)
ax[0].set_title("Original")
ax[1].imshow(grayscale, cmap=plt.cm.gray)
ax[1].set_title("Grayscale")

fig.tight_layout()
plt.show()

binary = grayscale < THRESHOLD

fig, axes = plt.subplots(ncols=3, figsize=(8, 2.5))
ax = axes.ravel()
ax[0] = plt.subplot(1, 3, 1)
ax[1] = plt.subplot(1, 3, 2)
ax[2] = plt.subplot(1, 3, 3, sharex=ax[0], sharey=ax[0])

ax[0].imshow(grayscale, cmap=plt.cm.gray)
ax[0].set_title('Original')
ax[0].axis('off')

ax[1].hist(grayscale.ravel(), bins=256)
ax[1].set_title('Histogram')
ax[1].axvline(THRESHOLD, color='r')

ax[2].imshow(binary, cmap=plt.cm.gray)
ax[2].set_title('Thresholded')
ax[2].axis('off')

plt.show()


cmap2 = matplotlib.colors.ListedColormap(['none', 'red'])


mask_file = os.path.join("HistoQuant/seg/seg/", "MASK_"+file+".png")
from skimage import io
mask = io.imread(mask_file)

img2 = plt.imshow(mask)
img3 = plt.imshow(binary, cmap=cmap2)
plt.show()

In [ ]:
ret1, thresh = cv2.threshold(cells, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
thresh

In [ ]:
#Video Playlist: https://www.youtube.com/playlist?list=PLHae9ggVvqPgyRQQOtENr6hK0m1UquGaG

"""
https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_watershed/py_watershed.html
This code performs cell counting and size distribution analysis and dumps results into a csv file.
It uses watershed segmentation for better segmentation, separating touching nuclei.
"""

import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from skimage import measure, color, io
from skimage.filters import threshold_otsu

filename = os.path.join("HistoQuant/raw", file)
img = io.imread(filename)
grayscale = rgb2gray(img)
thresh_ratio = threshold_otsu(grayscale)
print("Threshold: ", thresh_ratio)
# thresh = grayscale < thresh_ratio
thresh = grayscale < 0.25

pixels_to_um = calibration # 1 pixel = 454 nm (got this from the metadata of original image)

# #Threshold image to binary using OTSU. ALl thresholded pixels will be set to 255
# ret1, thresh = cv2.threshold(cells, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
# print("Treshold: ", ret1)


# Morphological operations to remove small noise - opening
#To remove holes we can use closing
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh * np.uint8(255),cv2.MORPH_OPEN,kernel, iterations = 2)

from skimage.segmentation import clear_border
opening = clear_border(opening) #Remove edge touching grains
plt.imshow(opening, cmap='gray') #This is our image to be segmented further using watershed
print("Clear Borders")
plt.show()
#Check the total regions found before and after applying this. 

#STEP 1: Sude background 
#Now we know that the regions at the center of cells is for sure cells
#The region far away is background.
#We need to extract sure regions. For that erode a few times. 
#But we have cells touching, so erode alone will not work. 
#To separate touching objects, the best approach would be distance transform and then thresholding.

# let us start by identifying sure background area
# dilating pixes a few times increases cell boundary to background. 
# This way whatever is remaining for sure will be background. 
#The area in between sure background and foreground is our ambiguous area. 
#Watershed should find this area for us. 
sure_bg = cv2.dilate(opening,kernel,iterations=10)
plt.imshow(sure_bg, cmap='gray') #Dark region is our sure background
print("Sure BG")
plt.show()
# Finding sure foreground area using distance transform and thresholding
#intensities of the points inside the foreground regions are changed to 
#distance their respective distances from the closest 0 value (boundary).
#https://www.tutorialspoint.com/opencv/opencv_distance_transformation.htm
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
plt.imshow(dist_transform, cmap='gray') #Dist transformed img. 
print("Dist Transform")
plt.show()

#Let us threshold the dist transform by starting at 1/2 its max value.
print(dist_transform.max()) #gives about 21.9
ret2, sure_fg = cv2.threshold(dist_transform,0.25*dist_transform.max(),255,0)
plt.imshow(sure_fg, cmap='gray')
print("Sure FG")
plt.show()
#Later you realize that 0.25* max value will not separate the cells well.
#High value like 0.7 will not recognize some cells. 0.5 seems to be a good compromize

# Unknown ambiguous region is nothing but bkground - foreground
sure_fg = np.uint8(sure_fg)  #Convert to uint8 from float
unknown = cv2.subtract(sure_bg,sure_fg)
plt.imshow(unknown, cmap='gray')
print("Unkown Regions")
plt.show()
#Now we create a marker and label the regions inside. 
# For sure regions, both foreground and background will be labeled with positive numbers.
# Unknown regions will be labeled 0. 
#For markers let us use ConnectedComponents. 
#Connected components labeling scans an image and groups its pixels into components 
#based on pixel connectivity, i.e. all pixels in a connected component share 
#similar pixel intensity values and are in some way connected with each other. 
#Once all groups have been determined, each pixel is labeled with a graylevel 
# or a color (color labeling) according to the component it was assigned to.
ret3, markers = cv2.connectedComponents(sure_fg)
plt.imshow(markers)
print("Markers")
plt.show()
#One problem rightnow is that the entire background pixels is given value 0.
#This means watershed considers this region as unknown.
#So let us add 10 to all labels so that sure background is not 0, but 10
markers = markers+10

# Now, mark the region of unknown with zero
markers[unknown==255] = 0
plt.imshow(markers)   #Look at the 3 distinct regions.
print("Three Distincts regions")
plt.show()
#Now we are ready for watershed filling. 
markers = cv2.watershed(img,markers)

#Let us color boundaries in yellow. 
#Remember that watershed assigns boundaries a value of -1
img[markers == -1] = [0,255,255]  

#label2rgb - Return an RGB image where color-coded labels are painted over the image.
img2 = color.label2rgb(markers, bg_label=0)

print('Overlay on original image')
plt.imshow(img)
plt.show()
print('Colored Grains')
plt.imshow(img2)
plt.show()

#####################################################################################
#Now, time to extract properties of detected cells
#Directly capturing props to pandas dataframe
    
props = measure.regionprops_table(markers, cells, 
                          properties=['label',
                                      'area', 'equivalent_diameter',
                                      'mean_intensity', 'solidity', 'orientation',
                                      'perimeter'])

import pandas as pd
df = pd.DataFrame(props)
print(df.head())

#To delete small regions...
df = df[df['area'] > 50]
print(df.head())

#######################################################
#Convert to micron scale
df['area_sq_microns'] = df['area'] * (pixels_to_um**2)
df['equivalent_diameter_microns'] = df['equivalent_diameter'] * (pixels_to_um)
print(df.head())

#df.to_csv('data/cast_iron_measurements.csv')

In [ ]:
df